# YAM 328 Makine Öğrenmesine Giriş
# Ödev 1
# 

# **Kuşların Kemiklerine göre yaşadıkları Habitatlar!**

In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import seaborn as sns
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



# **-Veri ön İşleme-**
# Veri setini seçiyoruz

In [ ]:
data = pd.read_csv('../input/birds-bones-and-living-habits/bird.csv')
data

# Veri setindeki kayıp verilerin sayısını gösterip siliyoruz.

In [ ]:
x=data.isnull().sum()
data.dropna(inplace=True)
data.drop(columns='id', inplace=True) #gereksiz gördüğüm bu kolonu da siliyorum
print(x)
data.info()




# **-Özellik Seçimi-**
# A-)Normalizasyon uyguluyoruz

In [ ]:
def normalization(data, col):
    data[col] = [ (i-data[col].min())/(data[col].max()-data[col].min())
    for i in data[col]]
columns_to_normalization = ['ulnal','ulnaw', 'femw','huml','humw','feml','tibl','tibw','tarl','tarw'] #
for column in columns_to_normalization:
    normalization(data, column)
data

# B-)Temel bileşen analizi

In [ ]:
def drawbarbox(d, x_col, y_col):
    plt.figure(figsize = (10,4))
    sns.barplot(x = data[x_col], y=data[y_col])
    plt.show()
def drawlineplot(data,x_col,y_col):
    plt.figure(figsize = (10,4))
    sns.lineplot(x = data[x_col], y=data[y_col])
    plt.show()
columns_to_drawbarbox = ['type']
columns_to_drawlineplot = ['ulnal','ulnaw','feml','femw','tibl','tibw','tarl','tarw']
for column in columns_to_drawbarbox:
    drawbarbox(data, column, 'huml')

    #temel bileşen analizi dağılım
sc=StandardScaler()
x_sc=sc.fit_transform(data.iloc[:,:-1].values)

pca=PCA(n_components=6)
x_pc=pca.fit_transform(x_sc)

plt.figure(figsize=(10,7));
sns.scatterplot(x_pc[:,0],x_pc[:,1],hue=data['type']);




for column in columns_to_drawlineplot:
    drawlineplot(data, column, 'huml')

In [ ]:
feature_columns = ['huml', 'humw', 'ulnal', 'ulnaw', 'feml', 'femw', 'tibl', 'tibw', 'tarl', 'tarw']
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(data[feature_columns])

explained_variance = pd.DataFrame({"evr": pca.explained_variance_ratio_, "evrc": pca.explained_variance_ratio_.cumsum()}, 
                                  index=pd.Index(["pc{:d}".format(i) for i in np.arange(1, len(feature_columns) + 1)], name="principle components"))
_, ax = plt.subplots(figsize=(8, 4))
explained_variance.evrc.plot(kind="line", color="#ee7621", ax=ax, linestyle="-", marker="h")
explained_variance.evr.plot(kind="bar", ax=ax, color="#00304e", alpha=0.8, rot=0)
ax.set_title("Explained Variance Ratio of Principle Components", fontsize=15)
ax.set_ylim([0.0, 1.1])

for x, y in zip(np.arange(0, len(explained_variance.evrc)), explained_variance.evrc):
    ax.annotate("{:.1f}%".format(y * 100.0), xy=(x-0.2, y+0.03), fontsize=10)

for x, y in zip(np.arange(1, len(explained_variance.evr)), explained_variance.evr[1:]):
    ax.annotate("{:.1f}%".format(y * 100.0), xy=(x-0.15, y+0.02), fontsize=15)

In [ ]:
pcs = pca.transform(data[feature_columns])

pc1 = pd.DataFrame({"pc1": pcs[:,0], "ecological_group": data.type})

_, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 4))
for g in pc1.ecological_group.unique():
    tmp = pc1[pc1.ecological_group == g]["pc1"].to_frame()
    tmp.columns=[g]
    tmp.plot(kind="kde", ax=ax)
    
_ = ax.set_title("KDE plots of 1st principle component for each ecological group", fontsize=15)

# covariance hesaplama


In [ ]:
cleaned = data.dropna()
cleaned_y = cleaned['type']
cleaned_x = cleaned.drop(['type'], axis=1)
cov_matrix = np.cov(cleaned_x.T)
cov_matrix




# eigenvalue hesaplama


In [ ]:
eig_values, eig_vectors = np.linalg.eig(cov_matrix)
print("Eigen Values of dataset: ")
print(eig_values)
print("---------------------------------------------------")
print("Eigen vector of dataset: ")
print(eig_vectors)

# En büyük ilk M özdeğere karşılık gelen M adet öz vektör(eigenvector) bulunur.


In [ ]:
eig_sum = np.sum(eig_values)
data_eig = [(i / eig_sum)*100 for i in sorted(eig_values, reverse=True)]
data_fr = np.cumsum(data_eig)
data_fr

# Veriler M öz vektöre izdüşürülerek N boyuttan M boyuta indirgenmiş olur.

In [ ]:
sns.lineplot(y=data_fr, x=range(len(data_fr)))
sns.lineplot(y=99, x=range(len(data_fr)))
plt.title("Eigenvalue kümülatif frekansı")

# K-NN

In [ ]:
X = data.drop('type', axis = 1)
Y = data['type']
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size =0.4 ,random_state=4)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, Y_train)
#Y_pred = knn.predict(X_test)
knn.score(X_test, Y_test)
